<a href="https://colab.research.google.com/github/ANMOL2001A/Mario_game_bot/blob/main/Mario_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install transformers torch
import torch
from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import json
import tqdm

class ChatData(Dataset):
    def __init__(self, path, tokenizer):
        self.data = json.load(open(path, "r", encoding="utf-8"))
        self.dialogues = []

        for conv in self.data:
            for turn in conv["dialog"]:
                if "question" in turn and "answer" in turn:
                    dialogue = f"<user>: {turn['question']} <bot>: {turn['answer']}"
                    self.dialogues.append(dialogue)

        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        return self.dialogues[idx]


def infer(inp, model, tokenizer, device, temperature=1.0):
    inp = f"<startofstring> {inp} <bot>:"
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a, max_length=100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, temperature=temperature)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output



def train(chatData, model, optim, device):
    epochs = 35

    for epoch in range(epochs):
        data_loader = DataLoader(chatData, batch_size=64, shuffle=True)
        for batch in tqdm.tqdm(data_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            X = chatData.tokenizer(batch, max_length=40, truncation=True, padding="max_length", return_tensors="pt")
            X_ids = X["input_ids"].to(device)
            X_mask = X["attention_mask"].to(device)
            optim.zero_grad()
            outputs = model(X_ids, attention_mask=X_mask, labels=X_ids)
            loss = outputs.loss
            loss.backward()
            optim.step()

        torch.save(model.state_dict(), "model_state2.pt")

        # Print example inference at the end of each epoch
        example_input = "Tell me about Mario."
        print(infer(example_input, model, chatData.tokenizer, device))

# Set up tokenizer and model
tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
model = OpenAIGPTLMHeadModel.from_pretrained("openai-gpt")
# Add special tokens if needed
tokenizer.add_special_tokens({"pad_token": "<pad>", "bos_token": "<startofstring>", "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])
model.resize_token_embeddings(len(tokenizer))

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Set up DataLoader and optimizer
chatData = ChatData("/content/Mario_question_answering_dataset.json", tokenizer)
optim = Adam(model.parameters(), lr=1e-3)

# Train the model
train(chatData, model, optim, device)


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
Epoch 1/35: 100%|██████████| 1/1 [00:00<00:00,  4.87it/s]


tell me about mario. <bot>: user > mario mario is known for mario s favorite hobby mario enjoys mario's favourite treasure s mario hunts mario > s mythical treasure mario accumus is reputed mario possesses mario quests mario resorts mario treasure hunts idi mario territories mario tains mario treasures mario owns mario hangs mario kong s legendary treasure > > is mario romano mario godfather mario has reputed reputed mythical mythical pirate mario mythical mario legend mario sells mario adventures mario exploits mario loves mario reputed s reputed treasure is idi s prized treasure kong


Epoch 2/35: 100%|██████████| 1/1 [00:00<00:00, 13.39it/s]


tell me about mario. <bot>: user user. user is a user that is known for his ability in the land. he is one of the most powerful and and.. and, and as a being of great power and strength and power. a power that can be seen in all things and have the power to do great and wondrous things. it is the kind and most beautiful of creatures. they are a living, living being that are the true form of life. their kind, as well as


Epoch 3/35: 100%|██████████| 1/1 [00:00<00:00, 12.97it/s]


...

tell me about mario. <bot>: mario is a fictional character and the iconic hero of the mario franchise. he's a plumber from the mushroom kingdom, known for his red cap and is often seen assisting mario on his adventures by providing transportation items to the adventures of princess peach and treasures. despite not being the help in his greed, is the management of rosalates him in the schemes of his schemes. is. me me him is to his kingdom. in him,. to me is,


Epoch 34/35: 100%|██████████| 1/1 [00:00<00:00, 11.04it/s]


tell me about mario. <bot>: mario is a fictional character and the iconic hero of the mario franchise. he's a plumber from the mushroom kingdom, known for his red cap and is often seen assisting mario on his schemes to save princess peach and his adventures by providing helpful transportation in his quests to the unique adventures. despite not being a'donkey kong'me him, is the abilities of his abilities in the'are a brave adventures, serves as a hero, to putting him to his


Epoch 35/35: 100%|██████████| 1/1 [00:00<00:00, 11.73it/s]


tell me about mario. <bot>: mario is a fictional character and the iconic hero of the mario franchise. he's a plumber from the mushroom kingdom, known for his red cap and is often seen assisting mario on his adventures by providing helpful transportation in his schemes to save princess peach and treasures. despite not being able to the management in the schemes of adventures, is the donkey kong't'c'me him. while he has me his quests by his abilities to a'are a is


In [ ]:
print("Infer from model:")
while True:
    inp = input()
    print(infer(inp, model, tokenizer, device,temperature=1.0))

Infer from model:
Tell me about Mario.
tell me about mario. <bot>: mario is a fictional character and the iconic hero of the mario franchise. he's a plumber from the mushroom kingdom, known for his red cap and is often seen assisting mario on his adventures by providing helpful transportation in his schemes to save princess peach and treasures. despite not being able to the management in the schemes of adventures, is the donkey kong't'c'me him. while he has me his quests by his abilities to a'are a is
What does Mario do?
what does mario do? <bot>: mario does is a skilled plumber, but he's best known for his adventures as a hero. he embarks on various quests to save princess peach and princess toadstool the mushroom kingdom, sometimes putting him in the care of rosalina. despite not being able to me power - up, to tell. in his free. <bot>: rosalments, is the help. to is. is me me is, and is to : me., in me him is
What is Mario's favorite power-up?
what is mario's favorite power - up? <b